In [1]:
import numpy as np
import plotly.express as px

from losses import BinaryCrossentropy
from optimizers import SGD_with_decay, Adam
from cifar_10_dataset_loading import load_cifar_10
from layers import Convolutional, Linear, Relu, Sigmoid, Flatten, Softmax, MaxPool, Layer

## Setup

### Data extraction

In [2]:
x_train, y_train, x_test, y_test = load_cifar_10()

In [3]:
classes = y_train.argmax(axis=1)

In [4]:
# Change range from [0, 255] to [-1, 1]
x_train = x_train / 255  
x_train.dtype

dtype('float64')

In [5]:
IMGS_IDX = [0, 351, 5673, 5494, 32, 55, 66, 776, 564]
x = x_train[IMGS_IDX]
y = y_train[IMGS_IDX]
px.imshow(x, facet_col=0, facet_col_wrap=4)

### Model declaration

In [6]:
def create_nn() -> list[Layer]:
    return [
        Convolutional((10, 5, 5, 3)),
        Relu(),
        MaxPool((2, 2)),
        Convolutional((10, 3, 3, 10)),
        Relu(),
        MaxPool((2, 2)),
        Flatten(),
        Linear(360, 64),
        Relu(),
        Linear(64, y.shape[1]),
        Softmax(),
    ]

## Training

In [7]:
training_stats = (
    Adam(
        starting_lr=0.01,
        lr_decay=0.0005,
        momentum_weight=0.9,
        ada_grad_weight=0.9,
    )
    .optimize_nn(
        create_nn(),
        x,
        y,
        epochs=100,
        batch_size=128,
        loss=BinaryCrossentropy(),
    )
)

Output()

In [8]:
fig = (
    px.scatter(
        training_stats.melt(["iteration"], ["loss",  "accuracy", "learning_rate"]), 
        x="iteration",
        y="value",
        color="variable",
        facet_row="variable",
        title="Training Metrics Over epochs",
        height=700,
    )
    .update_yaxes(matches=None)
)
fig.show()

In [39]:
N_SAMPLES = 100
nn = create_nn()
optimizer = Adam(
    starting_lr=0.01,
    lr_decay=0.0001,
    momentum_weight=0.99,
    ada_grad_weight=0.99,
)
training_stats = (
    optimizer
    .optimize_nn(
        nn,
        x_train[:N_SAMPLES],
        y_train[:N_SAMPLES],
        epochs=200,
        batch_size=128,
        loss=BinaryCrossentropy(),
    )
)

Output()

In [40]:
fig = (
    px.scatter(
        training_stats.melt(["iteration"], ["loss",  "accuracy", "learning_rate"]), 
        x="iteration",
        y="value",
        color="variable",
        facet_row="variable",
        title="Training Metrics Over epochs",
        height=700,
    )
    .update_yaxes(matches=None)
)
fig.show()